# Sorting Protein Mutations Using STRIDE

This notebook divides the dataset into seperate datasets depended on a mutations secondary structure assignment.

In [2]:
# import statements
import os
import numpy as np
import pandas as pd
import requests
from Bio import SeqIO
from io import StringIO
import Bio.PDB.Polypeptide

In [3]:
# setting jupyter notebook viewing options
max_rows = 1000
max_cols = 1000
pd.set_option("display.max_rows", max_rows, "display.max_columns", max_cols)

### Methods Used to Format Data

Formatting protein sequence into form for machine learning:

In [4]:
# parameters:
#      "uniprot_id" - string representing uniprot id of desired protein.
# This method uses a given uniprot id to query the uniprot data and 
# return a string respresention of the protein sequence. 
# E.g. MADIT
def get_protein_seq(uniprot_id):
    
    # importing fasta file from uniprot.org and getting protein sequence
    # taken from StackOverflow: 
    # https://stackoverflow.com/questions/52569622/protein-sequence-from-uniprot-protein-id-python
    url = "http://www.uniprot.org/uniprot/"
    complete_url = url + uniprot_id + ".fasta"
    response = requests.post(complete_url)
    data =''.join(response.text)
    sequence =StringIO(data)
    protein_seq=list(SeqIO.parse(sequence,'fasta'))

    # protein sequence as string (single-letter amino acids)
    string_seq = str(protein_seq[0].seq)
    
    # protein sequence w/ three-letter convention
    protein_seq = get_expanded_seq(string_seq)
    return protein_seq

Expanding protein sequence (1 letter AA -> 3 letter AA):

In [5]:
# parameter:
#      "seq" - string representing protein sequence in 1-letter convention.
# This method takes protein sequence string with 1-letter convention and returns
# a protein sequence with 3-letter convention.
# E.g. ADE -> ALA ASP GLU
def get_expanded_seq(seq):
    expanded_list = []
    split_seq = list(seq)
    for letter in split_seq:
        three_letter_abbr = Bio.PDB.Polypeptide.one_to_three(letter)
        expanded_list.append(three_letter_abbr)
    exanded_string = " ".join(expanded_list)
    return(exanded_string)

Returning index range of protein domain within protein:

In [6]:
# parameters: 
#      "full_protein_split" - list of amino acids in full protein in 3 letter convention.
#                             E.g. ["ALA", "GLY", "TYR"]
#      "domain_split" - list of amino acids in protein domain in 3 letter convention.
#                       E.g. ["ALA", "GLY"]
# This method prints the index of the given domain within the given protein.
# Starting value is inclusive and the ending value is exclusive. 
# E.g. [(0, 3)]
def get_index_range(full_protein_split, domain_split):
    indexes = []
    for i in range(len(full_protein_split)):
        if full_protein_split[i:i+len(domain_split)] == domain_split:
            indexes.append((i, i+len(domain_split)))
    print(indexes)
    indexes.clear()

Get variant in mutation-position form from wild-type-position-mutation form: (E.g. G126A -> 126ALA)

In [24]:
# parameter: 
#      "split_mutation_column" - list of mutations, split by comma if there are multiple.
# This method returns a list with wild-type residue (first letter) from variant.
def get_wild_type(split_mutation_column):
    wild_type_list = []
    w_letters = []
    for string in split_mutation_column:
        if "wild-type" in string[0]:
            wild_type = "wild_type"
        elif "-" in string[0] or len(string) == 0:
            wild_type = np.nan
        else:
            for val in string:
                mutation_name = val.strip(" ")
                w_letters.append(mutation_name[0])
                wild_type = ",".join(w_letters)
        wild_type_list.append(wild_type)
        w_letters.clear()
    return wild_type_list


# parameter: 
#      "split_mutation_column" - list of mutations, split by comma if there are multiple.
# This method returns a list with mutation residue (last letter) from variant.
def get_mutation_type(split_mutation_column):
    mutation_list = []
    m_letters = []
    for string in split_mutation_column:
        if "wild-type" in string[0]:
            mutation = "wild-type"
        elif "-" in string[0] or len(string) == 0:
            mutation = np.nan
        else:
            for val in string:
                mutation_name = val.strip(" ")
                m_letters.append(mutation_name[-1])
                mutation = ",".join(m_letters)
        mutation_list.append(mutation)
        m_letters.clear()
    return mutation_list


# parameter: 
#      "split_mutation_column" - list of mutations, split by comma if there are multiple.
# This method returns a list with the position of mutation (number) from variant.
def get_position(split_mutation_column):
    position_list = []
    p_letters = []
    for string in split_mutation_column:
        if "wild-type" in string[0]:
            position = "wild-type"
        elif "-" in string[0] or len(string) == 0:
            position = np.nan
        else:
            for val in string:
                mutation_name = val.strip(" ")
                p_letters.append(mutation_name[1:-1])
                position = ",".join(p_letters)
        position_list.append(position)
        p_letters.clear()
    return(position_list)

# parameter:
#      "df" - dataframe of protein data with "MUTATED_RES" and "POSITION" columns.
# This method returns a list with the correctly formatted variant (mutation-position form).
def get_mutations_names_list(df):
    formatted_list = []
    expanded_abbv = []
    for mutation, position in zip(df["MUTATED_RES"], df["POSITION"]):
        split_mutations = mutation.split(",")
        split_positions = position.split(",")
        if "wild-type" in split_mutations[0].lower() or "wild-type" in split_positions[0].lower():
            abbv_names = "WT"
        else:  
            for mut, pos in zip(split_mutations, split_positions):
                three_letter_mut = Bio.PDB.Polypeptide.one_to_three(mut.upper())
                position = str(int(pos))
                combined_name = position + three_letter_mut
                expanded_abbv.append(combined_name)
                abbv_names = ", ".join(expanded_abbv)
        expanded_abbv.clear()
        formatted_list.append(abbv_names)
    return(formatted_list)

Splits positions in intermediary "POSITION" column to help remove mutations with a certain position

In [25]:
# Parameters:
#      "df" - protein data dataframe with "POSITION" column 
# This method takes the position column in the dataframe and splits it in order
# to help remove or keep mutatations depending on their position.
def get_positions_split(df):
    position_list_split = []

    for item in df["POSITION"]:
        item = item.split(",") # splits positions into list
        int_item = [int(i) for i in item]
        position_list_split.append(int_item)
    
    return position_list_split

Getting Secondary Structure assignmnt from STRIDE file

In [26]:
# Parameters: 
#      "stride file" - stride file of protein
#      "is_sec_struc" - list of boolean values for each secondary structure value
#                       if it is, true, else false
# returns list of boolean values indicating if position is secondary strcuture or not
def get_sec_struc_boolean(stride_file):
    is_sec_struc = []
    sec_struc_assign = []

    for line in stride_file:
        if line.startswith('ASG'):
            split_line = line.split();
            sec_struc_assign.append(split_line[5])

    for sec_struc in sec_struc_assign:
        if (sec_struc =='C' or sec_struc =='T'):
            is_sec_struc.append(False)
        else:
            is_sec_struc.append(True)
            
    return is_sec_struc

Getting Dataset of Mutations within Domain in PDB File

In [27]:
# Parameters:
#      "orig_df" - 
#      "start" -
#      "end" - 
#      "not_included_list"
# This method does even more helpful stuff
def get_domain_dataset(orig_df, start, end, not_included_list):
    in_domain_list = []
    
    for val in orig_df["positions_split"]:
        for position in val:
            if not_included_list.count(position - start) == 0: # if value is not in the list of values to exclude
                if position >= start and position < end:
                    in_domain = True
                else:
                    in_domain = False
            else:
                in_domain = False
        in_domain_list.append(in_domain)
    
    orig_df['in_domain'] = in_domain_list
    # print(in_domain_list)
    condition = orig_df['in_domain'] == True
    rows = orig_df.loc[condition, :]
    
    in_domain_df = pd.DataFrame(columns=orig_df.columns)
    in_domain_df = in_domain_df.append(rows, ignore_index=True)
    in_domain_df = in_domain_df.drop(['in_domain'], axis=1)
    return in_domain_df

Getting Dataset of Mutations _in_ Secondary Structures

In [34]:
# Parameters:
# - orig_df: original dataframe with all mutations and "positions_split" column which has mutation positions in split list
#            as ints
# - sec_st_df: new dataframe with all rows that have mutations in the secondary structure of protein
# - mixed_df: new dataframe with all rows that have mutations in both in and out of the secondary stucture of the protein
# - start: (inclusive) index where the domain of the protein in PDB file starts
# - end: (inclusive) index where the domain of the protein in PDB file ends
def get_ss_dataset(orig_df, bool_ss_list, domain_start_index):
    
    has_sec_str = []
    
    for val in orig_df["positions_split"]:
        # list of boolean values that are true if positions are sec. strc.
        all_pos_sec_struc = []
        
        for position in val:
            # print(position - domain_start_index)
            # print(str(position) + " " + str(domain_start_index))
            if (bool_ss_list[position - domain_start_index] == False):
                all_pos_sec_struc.append(False)
            else:
                all_pos_sec_struc.append(True)
        
        # all pos sec struc should match val list
        # if there's a value in all_pos_sec_struc that's false, append false
        # otherwise, append true
        # print("val")
        # print(val)
        # print("bool")
        # print(all_pos_sec_struc)
        if (all_pos_sec_struc.count(False) == 0):
            has_only_sec_str = True
        else:
            has_only_sec_str = False
        
        # print(has_only_sec_str)
        has_sec_str.append(has_only_sec_str)
        all_pos_sec_struc.clear()
        
        
    # print(len(has_sec_str)) # should match dataframe length
    orig_df['has_sec_str'] = has_sec_str
    
    condition = orig_df['has_sec_str'] == True
    rows = orig_df.loc[condition, :]
    
    sec_str_df = pd.DataFrame(columns=orig_df.columns)
    sec_str_df = sec_str_df.append(rows, ignore_index=True)
    sec_str_df = sec_str_df.drop(['has_sec_str'], axis=1)
    orig_df = orig_df.drop(['has_sec_str'], axis=1)
    
    return sec_str_df

Getting Dataset of Mutations _not_ in Secondary Structures

In [36]:
def get_not_ss_dataset(orig_df, bool_ss_list, domain_start_index):
    is_not_sec_str = []
    
    for val in orig_df["positions_split"]:
        
        all_pos_sec_struc = []
        
        for position in val:
            # print(position - domain_start_index)
            # print(str(position) + " " + str(domain_start_index))
            if (bool_ss_list[position - domain_start_index] == False):
                all_pos_sec_struc.append(False)
            else:
                all_pos_sec_struc.append(True)
    
        
        if (all_pos_sec_struc.count(True) == 0):
            has_no_sec_str = True
        else:
            has_no_sec_str = False
        
        is_not_sec_str.append(has_no_sec_str)
        all_pos_sec_struc.clear()
        
    orig_df['is_not_sec_str'] = is_not_sec_str
     
    condition = orig_df['is_not_sec_str'] == True
    rows = orig_df.loc[condition, :]
    
    not_sec_str_df = pd.DataFrame(columns=orig_df.columns)
    not_sec_str_df = not_sec_str_df.append(rows, ignore_index=True)
    not_sec_str_df = not_sec_str_df.drop(['is_not_sec_str'], axis=1)
    orig_df = orig_df.drop(['is_not_sec_str'], axis=1)
    
    return not_sec_str_df

Writing formatted data to txt file:

In [74]:
# parameters:
#      "txt_name" - desired name of formatted txt file for network. E.g. "pab1"
#      "protein_seq" - string of protein sequence in 3 letter convention. E.g. ALA GLU TYR
#      "df" - dataframe with cleaned protein data. Must contain "variant" and "score" 
#             columns.
# This method cleans the protein data and formats it into a txt that can be processed by the 
# network. It also prints the name of the file out for reference.
def write_data_file(txt_name, protein_seq, df):
    file_name = txt_name + ".txt"
    print("Filename: " + file_name)
    
    datafile = open(file_name, "w+")
    datafile.write(protein_seq + "\n")
    for index in range(len(df)-1):
        datafile.write(df["variant"].iloc[index] + ": " + str(df["score"].iloc[index]) + "\n")
    datafile.write(df["variant"].iloc[len(df) - 1] + ": " + str(df["score"].iloc[len(df) - 1]))
    datafile.close()

Getting dataset of mutations that are in alpha helices:

Getting dataset of mutations that are in beta sheets:

## Pab1

Formatting Pab1 Data to Split Dataset into Values in Secondary Structure and NOT in Secondary Structure

In [26]:
# NOTE - stride files + jupyter notebook in winter dir.

In [1]:
pab1_stride_file = open('pab1_stride.txt', 'r')

In [3]:
pab1_ss_indexes = get_sec_struc_boolean(pab1_stride_file) # boolean list of secondary structure assignements

In [4]:
print(len(pab1_ss_indexes)) # <- domain is 75 AA long
print(pab1_ss_indexes)

75
[False, True, True, True, True, True, False, False, False, False, False, False, False, True, True, True, True, True, True, True, True, True, True, False, False, False, True, True, True, True, True, True, True, True, False, False, False, False, False, True, True, True, True, True, True, True, True, True, False, False, True, True, True, True, True, True, True, True, True, True, True, False, False, True, True, True, False, False, True, True, True, True, True, True, False]


In [5]:
# number of mutations not in secondary structure
count_false = pab1_ss_indexes.count(False)
print(count_false)

23


Limiting Number of Secondary Structure Mutations to Equal Number of Non-Secondary Structure Mutatations

- Pab1 has 23 Mutations not in Secondary Structure, so limiting Number of Secondary Structure Mutations to 23

In [8]:
# index of 23rd true

highest_true_index = [i for i, n in enumerate(pab1_ss_indexes) if n == True][23]
print(highest_true_index)
# need list of indices in secondary structure past this index in order to remove them from dataset

true_indices = [i for i,val in enumerate(pab1_ss_indexes) if val==True]
print(true_indices)

not_included_pab1 = [i for i in true_indices if i > 39]
print(not_included_pab1)

39
[1, 2, 3, 4, 5, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 26, 27, 28, 29, 30, 31, 32, 33, 39, 40, 41, 42, 43, 44, 45, 46, 47, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 63, 64, 65, 68, 69, 70, 71, 72, 73]
[40, 41, 42, 43, 44, 45, 46, 47, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 63, 64, 65, 68, 69, 70, 71, 72, 73]


### Sorting Pab1 Mutations Into 2 Datasets (w & w/o mutations)

In [9]:
# importing pab1 data from Gelman et al.
pab1_df1 = pd.read_csv("pab1.tsv.txt", sep="\t")
pab1_df = pab1_df1.dropna()
print(len(pab1_df))
print(pab1_df.columns)

40852
Index(['variant', 'num_mutations', 'score'], dtype='object')


In [10]:
# rounding score column to 2 decimal points
pab1_df["score"] = pab1_df["score"].round(6)
print(len(pab1_df))

# remove values with wildcard star thing cause idk what it means
pab1_df = pab1_df[pab1_df["variant"].str.contains("\*") == False]

# pab1_df = pab1_df.head(37600)
print(len(pab1_df))

40852
37710


In [12]:
# split variant name into wild-type, position, and mutation type
pab1_mut = pab1_df["variant"].str.split(",")
pab1_df["WILD_TYPE_RES"] = get_wild_type(pab1_mut)
pab1_df["MUTATED_RES"] = get_mutation_type(pab1_mut)
pab1_df["POSITION"] = get_position(pab1_mut)
pab1_df["variant"] = get_mutations_names_list(pab1_df)

In [13]:
# print(pab1_df.head)
print(pab1_df.columns)

Index(['variant', 'num_mutations', 'score', 'WILD_TYPE_RES', 'MUTATED_RES',
       'POSITION'],
      dtype='object')


Moving rows with Secondary Structure position into a different dataframe

In [16]:
pab1_df["positions_split"] = get_positions_split(pab1_df)
# print(pab1_df["positions_split"].tail(1000))

In [31]:
pab_in_domain_df = get_domain_dataset(pab1_df, 126, 201, not_included_pab1)

In [32]:
print(len(pab_in_domain_df))

22318


In [35]:
pab1_ss_df = get_ss_dataset(pab_in_domain_df, pab1_ss_indexes, 126)
print(len(pab1_ss_df))
# 5828 values

5828


In [48]:
# mini-dataset of 2880 values to compare mutations to non mutations:
pab1_ss_df_3000 = pab1_ss_df.sample(n=2880)
print(len(pab1_ss_df_3000))

2880


In [49]:
pab1_not_ss_df = get_not_ss_dataset(pab_in_domain_df, pab1_ss_indexes, 126)
print(len(pab1_not_ss_df))

3927


In [50]:
pab1_not_ss_df_3000 = pab1_not_ss_df.sample(n=2880)

### Putting Pab1 Datasets into Files

In [42]:
protein_seq_pab1 = get_protein_seq("P04147")

In [43]:
split_pab1 = protein_seq_pab1.split()
print(len(split_pab1))

577


In [ ]:
# NOTE - 3000 vals is actually 2880

In [75]:
# write data to formatted txt file

write_data_file("pab1_MLformat_ss_3000_lim", protein_seq_pab1, pab1_ss_df_3000)
write_data_file("pab1_MLformat_not_ss_3000_lim", protein_seq_pab1, pab1_not_ss_df_3000)

Filename: pab1_MLformat_ss_3000_lim.txt
Filename: pab1_MLformat_not_ss_3000_lim.txt


## Bgl3

Formatting Bgl3 Data to Split Dataset into Values in Secondary Structure and NOT in Secondary Structure

In [52]:
bgl3_stride_file = open('bgl3_stride.txt', 'r')

In [53]:
bgl3_ss_indexes = get_sec_struc_boolean(bgl3_stride_file)

In [54]:
print(len(bgl3_ss_indexes))
print(bgl3_ss_indexes)

501
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, True, True, True, True, True, False, False, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, True, True, True, True, True, True, True, True, False, False, False, False, False, True, True, True, False, False, False, False, False, False, False, False, False, False, False, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, True, True, True, True, True, False, False, True, True, True, True, False, False, False, False, False, False, False, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, True, True, True, True, True, True, True, False, False, False, False, True, True, True, True, True, True, True, False, False, False, False, False, True, True, True, True, True, True

In [55]:
# number of mutations in secondary structure (True), and not in secondary structure (False)
count_false = bgl3_ss_indexes.count(False)
print(count_false)

count_true = bgl3_ss_indexes.count(True)
print(count_true)

229
272


In [56]:
# index of 416 true

highest_true_index = [i for i, n in enumerate(bgl3_ss_indexes) if n == True][229]
print(highest_true_index)
# need list of indices past this index

true_indices = [i for i,val in enumerate(bgl3_ss_indexes) if val==True]
# print(true_indices)

not_included_bgl3 = [i for i in true_indices if i > highest_true_index]
# print(not_included_bgl3)

416


In [57]:
# importing bgl3 data from Gelman et al.
bgl3_df1 = pd.read_csv("bgl3.tsv.txt", sep="\t")
bgl3_df = bgl3_df1.dropna()
print(len(bgl3_df))
print(bgl3_df.columns)

26653
Index(['variant', 'num_mutations', 'inp', 'sel', 'score'], dtype='object')


In [58]:
# rounding score column to 6 decimal points
bgl3_df["score"] = bgl3_df["score"].round(6)
print(len(bgl3_df))

# remove values with wildcard star
bgl3_df = bgl3_df[bgl3_df["variant"].str.contains("\*") == False]
# bgl3_df = bgl3_df.head(25600)
print(len(bgl3_df))

26653
25737


In [60]:
# bgl3 protein sequence
string_seq = "MVPAAQQTAMAPDAALTFPEGFLWGSATASYQIEGAAAEDGRTPSIWDTYARTPGRVRNGDTGDVATDHYHRWREDVALMAELGLGAYRFSLAWPRIQPTGRGPALQKGLDFYRRLADELLAKGIQPVATLYHWDLPQELENAGGWPERATAERFAEYAAIAADALGDRVKTWTTLNEPWCSAFLGYGSGVHAPGRTDPVAALRAAHHLNLGHGLAVQALRDRLPADAQCSVTLNIHHVRPLTDSDADADAVRRIDALANRVFTGPMLQGAYPEDLVKDTAGLTDWSFVRDGDLRLAHQKLDFLGVNYYSPTLVSEADGSGTHNSDGHGRSAHSPWPGADRVAFHQPPGETTAMGWAVDPSGLYELLRRLSSDFPALPLVITENGAAFHDYADPEGNVNDPERIAYVRDHLAAVHRAIKDGSDVRGYFLWSLLDNFEWAHGYSKRFGAVYVDYPTGTRIPKASARWYAEVARTGVLPTAGDPNSSSVDKLAAALEHHHHHH"

In [61]:
print(len(string_seq))

501


In [62]:
protein_seq_bgl3 = get_expanded_seq(string_seq)
print(protein_seq_bgl3)

MET VAL PRO ALA ALA GLN GLN THR ALA MET ALA PRO ASP ALA ALA LEU THR PHE PRO GLU GLY PHE LEU TRP GLY SER ALA THR ALA SER TYR GLN ILE GLU GLY ALA ALA ALA GLU ASP GLY ARG THR PRO SER ILE TRP ASP THR TYR ALA ARG THR PRO GLY ARG VAL ARG ASN GLY ASP THR GLY ASP VAL ALA THR ASP HIS TYR HIS ARG TRP ARG GLU ASP VAL ALA LEU MET ALA GLU LEU GLY LEU GLY ALA TYR ARG PHE SER LEU ALA TRP PRO ARG ILE GLN PRO THR GLY ARG GLY PRO ALA LEU GLN LYS GLY LEU ASP PHE TYR ARG ARG LEU ALA ASP GLU LEU LEU ALA LYS GLY ILE GLN PRO VAL ALA THR LEU TYR HIS TRP ASP LEU PRO GLN GLU LEU GLU ASN ALA GLY GLY TRP PRO GLU ARG ALA THR ALA GLU ARG PHE ALA GLU TYR ALA ALA ILE ALA ALA ASP ALA LEU GLY ASP ARG VAL LYS THR TRP THR THR LEU ASN GLU PRO TRP CYS SER ALA PHE LEU GLY TYR GLY SER GLY VAL HIS ALA PRO GLY ARG THR ASP PRO VAL ALA ALA LEU ARG ALA ALA HIS HIS LEU ASN LEU GLY HIS GLY LEU ALA VAL GLN ALA LEU ARG ASP ARG LEU PRO ALA ASP ALA GLN CYS SER VAL THR LEU ASN ILE HIS HIS VAL ARG PRO LEU THR ASP SER ASP ALA ASP ALA ASP 

In [63]:
split = protein_seq_bgl3.split()
print(len(split))

501


In [66]:
# split variant name into wild-type, position, and mutation type
bgl3_mut = bgl3_df["variant"].str.split(",")
bgl3_df["WILD_TYPE_RES"] = get_wild_type(bgl3_mut)
bgl3_df["MUTATED_RES"] = get_mutation_type(bgl3_mut)
bgl3_df["POSITION"] = get_position(bgl3_mut)
bgl3_df["variant"] = get_mutations_names_list(bgl3_df)

In [67]:
bgl3_df["positions_split"] = get_positions_split(bgl3_df)

In [ ]:
def get_domain_dataset(orig_df, start, end, not_included_list):
    in_domain_list = []
    
    for val in orig_df["positions_split"]:
        for position in val:
            if not_included_list.count(position - start) == 0: # if value is not in the list of values to exclude
                if position >= start and position < end:
                    in_domain = True
            else:
                in_domain = False
        in_domain_list.append(in_domain)
    
    orig_df['in_domain'] = in_domain_list
    # print(in_domain_list)
    condition = orig_df['in_domain'] == True
    rows = orig_df.loc[condition, :]
    
    in_domain_df = pd.DataFrame(columns=orig_df.columns)
    in_domain_df = in_domain_df.append(rows, ignore_index=True)
    in_domain_df = in_domain_df.drop(['in_domain'], axis=1)
    return in_domain_df

In [69]:
bgl3_in_domain_df = get_domain_dataset(bgl3_df, 0, 550, not_included_bgl3) # ending is larger than sequence length bc. all mutations inside

In [70]:
bgl3_ss_df = get_ss_dataset(bgl3_in_domain_df, bgl3_ss_indexes, 0)
print(len(bgl3_ss_df))
bgl3_ss_df_3000 = bgl3_ss_df.sample(n=2880)

In [73]:
bgl3_not_ss_df = get_not_ss_dataset(bgl3_in_domain_df, bgl3_ss_indexes, 0)
print(len(bgl3_not_ss_df))
bgl3_not_ss_df_3000 = bgl3_not_ss_df.sample(n=2880)

7537


In [77]:
# write data to formatted txt file

write_data_file("bgl3_MLformat_ss_3000_lim", protein_seq_bgl3, bgl3_ss_df_3000)
write_data_file("pab1_MLformat_not_ss_3000_lim", protein_seq_bgl3, bgl3_not_ss_df_3000)

Filename: bgl3_MLformat_ss_3000_lim.txt
Filename: pab1_MLformat_not_ss_3000_lim.txt


## Ube4B

In [78]:
ube4b_stride_file = open('ube4b_stride.txt', 'r')

In [79]:
ube4b_ss_indexes = get_sec_struc_boolean(ube4b_stride_file)

In [80]:
print(len(ube4b_ss_indexes))
print(ube4b_ss_indexes)

102
[False, False, False, False, False, False, False, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, True, True, True, False, False, False, False, True, True, False, False, False, True, True, True, True, False, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, True, True, False, False, False, False, True, True, False, False, False, False, False, False, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False]


In [81]:
# index of 23rd true

highest_true_index = [i for i, n in enumerate(ube4b_ss_indexes) if n == True][49]
print(highest_true_index)
# need list of indices past this index

true_indices = [i for i,val in enumerate(ube4b_ss_indexes) if val==True]
print(true_indices)

not_included_ube4b = [i for i in true_indices if i > highest_true_index]
# [x for x in a if x <= 5]
print(not_included_ube4b)

95
[7, 8, 9, 10, 11, 12, 13, 29, 30, 31, 32, 33, 38, 39, 43, 44, 45, 46, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 66, 67, 72, 73, 80, 81, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98]
[96, 97, 98]


In [84]:
# number of mutations in secondary structure (True), and not in secondary structure (False)
count_false = ube4b_ss_indexes.count(False)
print(count_false)

count_true = ube4b_ss_indexes.count(True)
print(count_true)

49
53


In [85]:
# importing Ube4b data from Gelman et al.
ube4b_df1 = pd.read_csv("ube4b.tsv.txt", sep="\t")
ube4b_df = ube4b_df1.dropna()
print(len(ube4b_df))
print(ube4b_df.columns)

98297
Index(['variant', 'num_mutations', 'score'], dtype='object')


In [86]:
# rounding score column to 6 decimal points
ube4b_df["score"] = ube4b_df["score"].round(6)
print(len(ube4b_df))

# remove values with wildcard star
ube4b_df = ube4b_df[ube4b_df["variant"].str.contains("\*") == False]
print(len(ube4b_df))

98297
91031


In [88]:
protein_seq_ube4b = get_protein_seq("Q9ES00")
split_entire = protein_seq_ube4b.split()
print(len(split_entire))

1173


In [90]:
string_seq = "IEKFKLLAEKVEEIVAKNARAEIDYSDAPDEFRDPLMDTLMTDPVRLPSGTVMDRSIILRHLLNSPTDPFNRQMLTESMLEPVPELKEQIQAWMREKQSSDH"
print(len(string_seq))

In [90]:
def get_expanded_seq(seq):
    expanded_list = []
    split_seq = list(seq)
    for letter in split_seq:
        three_letter_abbr = Bio.PDB.Polypeptide.one_to_three(letter)
        expanded_list.append(three_letter_abbr)
    exanded_string = " ".join(expanded_list)
    return(exanded_string)
    
protein_seq_ube4b_domain = get_expanded_seq(string_seq)
print(protein_seq_ube4b_domain)

ILE GLU LYS PHE LYS LEU LEU ALA GLU LYS VAL GLU GLU ILE VAL ALA LYS ASN ALA ARG ALA GLU ILE ASP TYR SER ASP ALA PRO ASP GLU PHE ARG ASP PRO LEU MET ASP THR LEU MET THR ASP PRO VAL ARG LEU PRO SER GLY THR VAL MET ASP ARG SER ILE ILE LEU ARG HIS LEU LEU ASN SER PRO THR ASP PRO PHE ASN ARG GLN MET LEU THR GLU SER MET LEU GLU PRO VAL PRO GLU LEU LYS GLU GLN ILE GLN ALA TRP MET ARG GLU LYS GLN SER SER ASP HIS


In [95]:
split = protein_seq_ube4b_domain.split()
print(len(split)) 
# correct sequence ->
print(split[3])

102
PHE


In [ ]:
# index of domain inside protein

indexes = []
for i in range(len(split_list)):
    if split_list[i:i+len(split)] == split:
        indexes.append((i, i+len(split)))
print(indexes)

In [1]:
ube4b_mut = ube4b_df["variant"].str.split(",")

ube4b_df["WILD_TYPE_RES"] = get_wild_type(ube4b_mut)
ube4b_df["MUTATED_RES"] = get_mutation_type(ube4b_mut)
ube4b_df["POSITION"] = get_position(ube4b_mut)

ube4b_df["variant"] = get_mutations_names_list(ube4b_df)
print(ube4b_df.columns)

NameError: name 'ube4b_df' is not defined

In [116]:
position_list_split = []

for item in ube4b_df["POSITION"]:
    item = item.split(",") # splits positions into list
    int_item = [int(i) for i in item]
    position_list_split.append(int_item)

var = position_list_split[0]
print(type(var[0]))

<class 'int'>


In [117]:
ube4b_df["positions_split"] = position_list_split

In [126]:
# ube4b_in_domain_df = get_domain_dataset(ube4b_df, 0, 1200)
ube4b_in_domain_df = get_domain_dataset(ube4b_df, 0, 2000, not_included_ube4b)
print(len(ube4b_in_domain_df))

81832


In [77]:
print(ube4b_df["positions_split"].head(1000))

0                [52, 63, 70, 73]
1                    [36, 52, 96]
2                        [16, 35]
3                     [2, 37, 52]
4                     [5, 52, 95]
5                    [37, 52, 98]
6                    [52, 70, 88]
7                    [24, 52, 69]
8                        [17, 70]
9                     [6, 52, 73]
10                  [18, 67, 101]
11                        [3, 50]
12                   [52, 63, 98]
13                    [9, 52, 55]
14                       [52, 70]
15                   [12, 37, 70]
16                [6, 15, 19, 52]
17                  [52, 96, 101]
18                    [8, 52, 98]
19                       [23, 52]
20                       [52, 70]
21                   [15, 23, 52]
22                  [27, 52, 101]
23                       [52, 70]
24                    [8, 52, 72]
25               [16, 50, 52, 70]
26                   [24, 52, 95]
27                    [3, 17, 52]
28                   [36, 52, 90]
29            

In [127]:
ube4b_ss_df = get_ss_dataset(ube4b_in_domain_df, ube4b_ss_indexes, 0)
print(len(ube4b_ss_df))

11450


In [128]:
ube4b_not_ss_df = get_not_ss_dataset(ube4b_in_domain_df, ube4b_ss_indexes, 0)

In [129]:
print(len(ube4b_not_ss_df))

20301


In [130]:
ube4b_ss_df_3000 = ube4b_ss_df.sample(n=2880)

In [131]:
ube4b_not_ss_df_3000 = ube4b_not_ss_df.sample(n=2880)

In [132]:
# write data to formatted txt file

file_name = "ube4b_MLformat_ss_3000_lim" + ".txt"
print("Filename: " + file_name)

datafile = open(file_name, "w+")
datafile.write(protein_seq_ube4b + "\n")
for index in range(len(ube4b_ss_df_3000)-1):
    datafile.write(ube4b_ss_df_3000["variant"].iloc[index] + ": " + str(ube4b_ss_df_3000["score"].iloc[index]) + "\n")
datafile.write(ube4b_ss_df_3000["variant"].iloc[len(ube4b_ss_df_3000)-1] + ": " + str(ube4b_ss_df_3000["score"].iloc[len(ube4b_ss_df_3000)-1]))
datafile.close()

Filename: ube4b_MLformat_ss_3000_lim.txt


In [133]:
# write data to formatted txt file

file_name = "ube4b_MLformat_not_ss_3000_lim" + ".txt"
print("Filename: " + file_name)

datafile = open(file_name, "w+")
datafile.write(protein_seq_ube4b + "\n")
for index in range(len(ube4b_not_ss_df_3000)-1):
    datafile.write(ube4b_not_ss_df_3000["variant"].iloc[index] + ": " + str(ube4b_not_ss_df_3000["score"].iloc[index]) + "\n")
datafile.write(ube4b_not_ss_df_3000["variant"].iloc[len(ube4b_not_ss_df_3000)-1] + ": " + str(ube4b_not_ss_df_3000["score"].iloc[len(ube4b_not_ss_df_3000)-1]))
datafile.close()

Filename: ube4b_MLformat_not_ss_3000_lim.txt
